## Tanımlama ve Bağıntılar

In [10]:
import numpy as np
import pandas as pd
from sympy import symbols, sqrt, diff, solve, Eq, integrate
from sympy import symbols, Function, Eq, dsolve, Symbol
from sympy import pi, exp, simplify
from astropy import units as u
from astropy.cosmology import Planck18

# Hubble sabiti Planck 2018'e göre
H0 = Planck18.H0   # 67.4 km/s/Mpc

# Gyr^-1'e çevir
hdeger = H0.to(1/u.Gyr) #0.0691967050869917 1 / Gyr0.0691967050869917 1 / Gyr

# Sembolik değişkenler
t, alpha, beta = symbols('t alpha beta')
c, c2, delta, lambda1, lambda2 = symbols('c c2 delta lambda1 lambda2')

a_t = Function('a')(t)  # a(t) fonksiyonu
C1 = Symbol('C1')  # C1'i sembol olarak tanımla (critical step!)

# q hangi değere eşit
tdeger = 13.8
qdeger = -0.57
#hdeger = 0.069

# Virgülden sonra kullanılaccak basamak sayısı
hassasiyet = 0.01
bs = 3

# Hubble tanımı
def h0(t, alpha, beta):
    return 1 / sqrt(2 * alpha * t + beta)

def h(t, alpha, beta):
    return (alpha*exp(alpha*t))/((1+beta)*(exp(alpha*t)-1))

# q = -1 + (1/H)'
def q(t, alpha, beta):
    return -1 + diff(1 / h(t, alpha, beta), t)

# Hubble = a'(t)/a(t)
def a(t, alpha, beta):
    d1 = Eq(h(t, alpha, beta), diff(a_t, t) / a_t)
    d2 = dsolve(d1,a_t).rhs
    return d2

def rho_b(t, alpha, beta, delta, c):
    return c * (2 * alpha * t + beta)**(-1+delta/2)

def rho_b_d(t, alpha, beta, delta, c):
    return diff(rho_b(t, alpha, beta, delta, c), t)

def rho_m(t, alpha, beta, delta, c, c2, lambda1, lambda2):
    return (192*(pi + (3*lambda2)/8)*(alpha*t + beta/2)**2*lambda1*c2**2*exp(-8*sqrt(2*alpha*t + beta)/alpha) 
         - 128*(pi + lambda2/4)*c*(pi + lambda2/8)*(alpha*t + beta/2)*(2*alpha*t + beta)**(delta/2) 
         - 3*lambda1*(sqrt(2*alpha*t + beta)*alpha*lambda2 + 32*(pi + lambda2/8)*(alpha*t + beta/2))) / \
        (2*(8*pi + lambda2)*(4*pi + lambda2)*(2*alpha*t + beta)**2)

def rho_m_d(t, alpha, beta, delta, c, c2, lambda1, lambda2):
    return diff(rho_m(t, alpha, beta, delta, c, c2, lambda1, lambda2), t)

def p(t, alpha, beta, delta, c2, lambda1, lambda2):
    return 12 * (c2**2 * (alpha*t + beta/2)**2 * exp(-8*sqrt(2*alpha*t + beta)/alpha) 
          + alpha*t/2 - sqrt(2*alpha*t + beta)*alpha/8 + beta/4) * lambda1 / ((4*pi + lambda2)*(2*alpha*t + beta)**2)

def p_d(t, alpha, beta, delta, c2, lambda1, lambda2):
    return diff(p(t, alpha, beta, delta, c2, lambda1, lambda2), t)

def omega(t, alpha, beta, delta, c, c2, lambda1, lambda2):
    return p(t, alpha, beta, delta, c2, lambda1, lambda2)/rho_b(t, alpha, beta, delta, c)
print(hdeger)

0.0691967050869917 1 / Gyr


In [2]:
print("*" * 60 + "\n")
baginti1 = solve(Eq(q(t,alpha,beta), qdeger),beta)
print(f"Basınç için alfa-beta bağıntısı (q = {qdeger}):")
print(f"b = {baginti1[0]}" + "\n")

print("*" * 20 + "\n")
baginti2 = solve(Eq(q(t, alpha, beta), 0), t)
print("Transit noktası için Alfa-Beta bağıntısı (q(t_tr) = 0):")
print(f"{tdeger} = {baginti2[0]}" + "\n")

print("*" *20 + "\n")
# Diferansiyel denklemi kur ve çöz
C1_solution = solve(Eq(a(t, alpha, beta), 1), C1)
print(f"C1 = {C1_solution[0]}" + "\n")
print("*" * 60)

************************************************************

Basınç için alfa-beta bağıntısı (q = -0.57):
b = 0.43*exp(alpha*t) - 1.0

********************

Transit noktası için Alfa-Beta bağıntısı (q(t_tr) = 0):
13.8 = log(beta + 1)/alpha

********************

C1 = -log(exp(alpha*t) - 1)/(beta + 1)

************************************************************


## $\alpha$ - $\beta$ - $c_1$ - $t_{tr}$ Değerleri

In [3]:
# AŞAMA 1: alpha ve beta değerlerini döngüyle oluştur ve filtrele
step1_list = []


# Transit nokta aralığı
t_aralik = [6.5,7.5]

alpha_aralik = [-10,10]
beta_aralik = [-50,50]

for alp_deg in np.arange(alpha_aralik[0] , alpha_aralik[1], hassasiyet):
    bet_deg = baginti1[0].subs({t: tdeger, alpha: alp_deg}).evalf()
    
    if beta_aralik[0] <= bet_deg <= beta_aralik[1]:
        c1_deg = C1_solution[0].subs({t: tdeger, alpha: alp_deg, beta: bet_deg}).evalf()
        q_raw = baginti2[0].subs({alpha: alp_deg, beta: bet_deg}).evalf()
    
        if q_raw.is_real and t_aralik[0] <= q_raw <= t_aralik[1]:
            step1_list.append({'alpha': alp_deg, 'beta': bet_deg, 'c1': c1_deg, 'transit': q_raw})

In [4]:
print("*" * 60)
print("🔹 Aşama 1 Sonuçları:", len(step1_list))
df1 = pd.DataFrame(step1_list).astype(float)
print(df1.head(45).to_string(index=False , float_format="{:.4f}".format))
print("*" * 60)

************************************************************
🔹 Aşama 1 Sonuçları: 2
 alpha   beta      c1  transit
0.1200 1.2525 -0.6411   6.7669
0.1300 1.5858 -0.6235   7.3079
************************************************************


## $\alpha$ - $\beta$ - $c_1$ - $t_{tr}$ - $c$ - $\delta$ Değerleri

In [5]:
# AŞAMA 2: c ve delta değerlerini alpha ve beta'ye bağlı olarak üret
step2_list = []

c_aralik = [0,2]
d_aralik = [0,1]

for item in step1_list:
    alp_deg = item['alpha']
    bet_deg = item['beta']
    c1_deg = item['c1']
    q_raw = item['transit']
    
    for c_deg in np.arange(c_aralik[0], c_aralik[1], hassasiyet*100):
        for d_deg in np.arange(d_aralik[0], d_aralik[1], hassasiyet*10):

            # Karanlık enerjinin büyük patlamadan sonra ilk yıllarındaki değeri
            rho_b_t_ilk = rho_b(t, alp_deg, bet_deg, d_deg, c_deg).subs({t: 2})
            # Karanlık enerjinin günümüzdeki değeri
            rho_b_t_son = rho_b(t, alp_deg, bet_deg, d_deg, c_deg).subs({t: tdeger})
            # Karanlık enerji grafiğinin başlangıçtaki değerinin türevi grafik azalan olduğundan negatif olması beklenir
            rho_b_d_ilk = rho_b_d(t, alp_deg, bet_deg, d_deg, c_deg).subs({t: 2})
            # Karanlık enerji grafiğinin günümüzdeki değerinin türevi grafik azalan olduğundan negatif olması beklenir
            rho_b_d_son = rho_b_d(t, alp_deg, bet_deg, d_deg, c_deg).subs({t: tdeger})
            if (rho_b_t_ilk.is_real and rho_b_t_son.is_real and c_deg != 0 and d_deg != 0):
                if (rho_b_t_ilk > 0 and rho_b_t_son > 0 and rho_b_d_ilk < 0 and rho_b_d_son < 0) and (rho_b_t_ilk > rho_b_t_son):
                    step2_list.append({
                        'alpha': alp_deg,
                        'beta': bet_deg,
                        'c1' : c1_deg,
                        'transit' : q_raw,
                        'c': c_deg,
                        'delta': d_deg,
                        'rho_b' : rho_b_t_ilk,
                        'rho_b_d' : rho_b_d_ilk
                    })

In [6]:
print("*" * 60)
print(" 🔹 Aşama 2 Sonuçları:", len(step2_list))
df2 = pd.DataFrame(step2_list).astype(float)
print(df2.head().to_string(index=False , float_format="{:.4f}".format))
print("*" * 60)

************************************************************
 🔹 Aşama 2 Sonuçları: 18
 alpha   beta      c1  transit      c  delta  rho_b  rho_b_d
0.1200 1.2525 -0.6411   6.7669 1.0000 0.1000 0.5933  -0.0781
0.1200 1.2525 -0.6411   6.7669 1.0000 0.2000 0.6098  -0.0760
0.1200 1.2525 -0.6411   6.7669 1.0000 0.3000 0.6268  -0.0738
0.1200 1.2525 -0.6411   6.7669 1.0000 0.4000 0.6443  -0.0714
0.1200 1.2525 -0.6411   6.7669 1.0000 0.5000 0.6622  -0.0688
************************************************************


## $\alpha$ - $\beta$ - $c_1$ - $t_{tr}$ - $c$ - $\delta$ - $c_2$ - $\lambda_1$ - $\lambda_2$ Değerleri

In [7]:
# AŞAMA 3: c2, lambda1 ve lambda2 değerlerini alpha,beta,c ve delta'ye bağlı olarak üret
step3_list = []

c2_aralik = [1,2]
l1_aralik = [-5,-3]
l2_aralik = [-5,-3]

for item in step2_list:
    alp_deg = item['alpha']
    bet_deg = item['beta']
    c1_deg = item['c1']
    q_raw = item['transit']
    c_deg = item['c']
    d_deg = item['delta']
    
    for c2_deg in np.arange(c2_aralik[0], c2_aralik[1], hassasiyet*100):
        for l1_deg in np.arange(l1_aralik[0], l1_aralik[1], hassasiyet*100):
            for l2_deg in np.arange(l2_aralik[0], l2_aralik[1], hassasiyet*100):
                
                if c2_deg != 0 and l1_deg != 0 and l2_deg != 0:
                    # Karanlık enerjinin büyük patlamadan sonra ilk yıllarındaki değeri
                    rho_m_t_ilk = rho_m(t, alp_deg, bet_deg, d_deg, c_deg,c2_deg,l1_deg,l2_deg).subs({t: 2})
                    # Karanlık enerjinin günümüzdeki değeri
                    rho_m_t_son = rho_m(t, alp_deg, bet_deg, d_deg, c_deg,c2_deg,l1_deg,l2_deg).subs({t: tdeger})
                    # Karanlık enerji grafiğinin başlangıçtaki değerinin türevi grafik azalan olduğundan negatif olması beklenir
                    rho_m_d_ilk = rho_m_d(t, alp_deg, bet_deg, d_deg, c_deg,c2_deg,l1_deg,l2_deg).subs({t: 2})
                    # Karanlık enerji grafiğinin günümüzdeki değerinin türevi grafik azalan olduğundan negatif olması beklenir
                    rho_m_d_son = rho_m_d(t, alp_deg, bet_deg, d_deg, c_deg,c2_deg,l1_deg,l2_deg).subs({t: tdeger})

                    if (rho_m_t_ilk > 0 and rho_m_t_son > 0 and rho_m_d_ilk < 0 and rho_m_d_son < 0):
                        if (rho_m_t_ilk > rho_m_t_son and rho_m_d_ilk < rho_m_d_son): 
                            
                            p_t_ilk = p(t, alp_deg, bet_deg, d_deg, c2_deg, l1_deg, l2_deg).subs({t: 2}).evalf()
                            p_t_son = p(t, alp_deg, bet_deg, d_deg, c2_deg, l1_deg, l2_deg).subs({t: tdeger}).evalf()
                            if (p_t_ilk < 0 and p_t_son < 0):
                                eos = omega(t, alp_deg, bet_deg, d_deg, c_deg, c2_deg, l1_deg, l2_deg).subs({t: tdeger}).evalf()
                                if (eos > -1.2 and eos < -0.8):
                                    step3_list.append({
                                        'alpha': alp_deg,
                                        'beta': bet_deg,
                                        'c1' : c1_deg,
                                        'transit' : q_raw,
                                        'c': c_deg,
                                        'delta': d_deg,
                                        'c2' : c2_deg,
                                        'Lambda_1': l1_deg,
                                        'Lambda_2': l2_deg,
                                        'Omega' : eos
                                    })

In [8]:
print("*" * 60)
print(" 🔹 Aşama 3 Sonuçları:", len(step3_list))
df3 = pd.DataFrame(step3_list).astype(float)
print(df3.head(20).to_string(index=False, float_format="{:.4f}".format))
print("*" * 60)

************************************************************
 🔹 Aşama 3 Sonuçları: 20
 alpha   beta      c1  transit      c  delta     c2  Lambda_1  Lambda_2   Omega
0.1200 1.2525 -0.6411   6.7669 1.0000 0.2000 1.0000   -4.0000   -4.0000 -1.1697
0.1200 1.2525 -0.6411   6.7669 1.0000 0.3000 1.0000   -4.0000   -4.0000 -1.0842
0.1200 1.2525 -0.6411   6.7669 1.0000 0.4000 1.0000   -4.0000   -5.0000 -1.1377
0.1200 1.2525 -0.6411   6.7669 1.0000 0.4000 1.0000   -4.0000   -4.0000 -1.0049
0.1200 1.2525 -0.6411   6.7669 1.0000 0.5000 1.0000   -5.0000   -4.0000 -1.1643
0.1200 1.2525 -0.6411   6.7669 1.0000 0.5000 1.0000   -4.0000   -5.0000 -1.0546
0.1200 1.2525 -0.6411   6.7669 1.0000 0.6000 1.0000   -5.0000   -4.0000 -1.0792
0.1200 1.2525 -0.6411   6.7669 1.0000 0.7000 1.0000   -5.0000   -5.0000 -1.1325
0.1200 1.2525 -0.6411   6.7669 1.0000 0.7000 1.0000   -5.0000   -4.0000 -1.0003
0.1200 1.2525 -0.6411   6.7669 1.0000 0.8000 1.0000   -5.0000   -5.0000 -1.0497
0.1300 1.5858 -0.6235   7.3079 1.0